In [1]:
!pip install dropconnect-tensorflow

  Created wheel for dropconnect-tensorflow: filename=dropconnect_tensorflow-0.0.3-py3-none-any.whl size=3356 sha256=1c2d43fbbfbd1c6fc2145232e4c43ca963164d2e184f2af8ad8621666a8b5623
  Stored in directory: /root/.cache/pip/wheels/0d/3a/21/c4ccb62b39f6405d6660edefe968d571aafcb54cdad0e12bd5
Successfully built dropconnect-tensorflow


In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, Input, ReLU, BatchNormalization, Flatten, MaxPool2D

from dropconnect_tensorflow import DropConnectDense, DropConnectConv2D

## Test DropConnectDense

In [ ]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
X = tf.keras.layers.Input(shape=(784,))
x = DropConnectDense(units=128, prob=0.2, activation="relu", use_bias=True)(X)
x = DropConnectDense(units=64, prob=0.5, activation="relu", use_bias=True)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
drop_connect_dense_2 (DropCo (None, 128)               100480    
_________________________________________________________________
drop_connect_dense_3 (DropCo (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Hyperparameters
batch_size=64
epochs=20


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/20
844/844 [==============================] - 6s 4ms/step - loss: 0.7680 - accuracy: 0.8009 - val_loss: 0.2890 - val_accuracy: 0.9252
Epoch 2/20
844/844 [==============================] - 3s 3ms/step - loss: 0.2975 - accuracy: 0.9161 - val_loss: 0.2124 - val_accuracy: 0.9437
Epoch 3/20
844/844 [==============================] - 3s 3ms/step - loss: 0.2368 - accuracy: 0.9328 - val_loss: 0.1813 - val_accuracy: 0.9507
Epoch 4/20
844/844 [==============================] - 3s 3ms/step - loss: 0.2035 - accuracy: 0.9421 - val_loss: 0.1571 - val_accuracy: 0.9587
Epoch 5/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1792 - accuracy: 0.9491 - val_loss: 0.1427 - val_accuracy: 0.9615
Epoch 6/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1608 - accuracy: 0.9545 - val_loss: 0.1353 - val_accuracy: 0.9637
Epoch 7/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1456 - accuracy: 0.9584 - val_loss: 0.1208 - val_accuracy: 0.9660
Epoch 

In [ ]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 0s - loss: 0.0876 - accuracy: 0.9730
Test loss = 0.08760864287614822, Test acc: 0.9729999899864197


## Test DropConnectConv2D

In [ ]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = np.expand_dims(x_train, axis=-1).astype('float32') / 255
x_test = np.expand_dims(x_test, axis=-1).astype('float32') / 255

In [ ]:
X = tf.keras.layers.Input(shape=(28, 28, 1))
x = DropConnectConv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', prob=0.1)(X)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)
x = DropConnectConv2D(filters=128, kernel_size=3, strides=(1, 1), padding='valid', prob=0.1)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)

x = Flatten()(x)
x = DropConnectDense(units=64, prob=0.3, activation="relu", use_bias=True)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
drop_connect_conv2d (DropCon (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
drop_connect_conv2d_1 (DropC (None, 11, 11, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 128)       512 

In [ ]:
# Hyperparameters
batch_size=64
epochs=10


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/10
844/844 [==============================] - 35s 10ms/step - loss: 0.2375 - accuracy: 0.9303 - val_loss: 0.0647 - val_accuracy: 0.9840
Epoch 2/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0637 - accuracy: 0.9812 - val_loss: 0.0474 - val_accuracy: 0.9848
Epoch 3/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0440 - accuracy: 0.9870 - val_loss: 0.0484 - val_accuracy: 0.9862
Epoch 4/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0332 - accuracy: 0.9898 - val_loss: 0.0426 - val_accuracy: 0.9867
Epoch 5/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0256 - accuracy: 0.9921 - val_loss: 0.0351 - val_accuracy: 0.9888
Epoch 6/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0212 - accuracy: 0.9936 - val_loss: 0.0442 - val_accuracy: 0.9877
Epoch 7/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0157 - accuracy: 0.9953 - val_loss: 0.0386 - val_accuracy: 0.9890
Epo

In [ ]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 1s - loss: 0.0388 - accuracy: 0.9880
Test loss = 0.038835447281599045, Test acc: 0.9879999756813049


# Test Wrapper

In [13]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Dense, Conv2D, Input, ReLU, BatchNormalization, Flatten, MaxPool2D, LSTM

from dropconnect_tensorflow import DropConnect

## Test Dense with DropConnect Wrapper

In [87]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

In [88]:
X = tf.keras.layers.Input(shape=(784,))
x = DropConnect(Dense(128, activation="relu"), prob=0.5)(X)
x = DropConnect(Dense(64, activation="relu"), prob=0.5)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [89]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 784)]             0         
_________________________________________________________________
drop_connect_19 (DropConnect (None, 128)               100480    
_________________________________________________________________
drop_connect_20 (DropConnect (None, 64)                8256      
_________________________________________________________________
dense_28 (Dense)             (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [90]:
# Hyperparameters
batch_size=64
epochs=20


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/20
844/844 [==============================] - 4s 4ms/step - loss: 0.7740 - accuracy: 0.7966 - val_loss: 0.2914 - val_accuracy: 0.9292
Epoch 2/20
844/844 [==============================] - 3s 4ms/step - loss: 0.3006 - accuracy: 0.9179 - val_loss: 0.2122 - val_accuracy: 0.9453
Epoch 3/20
844/844 [==============================] - 3s 4ms/step - loss: 0.2398 - accuracy: 0.9338 - val_loss: 0.1867 - val_accuracy: 0.9518
Epoch 4/20
844/844 [==============================] - 3s 4ms/step - loss: 0.2062 - accuracy: 0.9419 - val_loss: 0.1582 - val_accuracy: 0.9588
Epoch 5/20
844/844 [==============================] - 3s 4ms/step - loss: 0.1819 - accuracy: 0.9483 - val_loss: 0.1445 - val_accuracy: 0.9612
Epoch 6/20
844/844 [==============================] - 3s 3ms/step - loss: 0.1631 - accuracy: 0.9535 - val_loss: 0.1347 - val_accuracy: 0.9637
Epoch 7/20
844/844 [==============================] - 3s 4ms/step - loss: 0.1487 - accuracy: 0.9576 - val_loss: 0.1277 - val_accuracy: 0.9662
Epoch 

In [91]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 0s - loss: 0.0898 - accuracy: 0.9727
Test loss = 0.08980447798967361, Test acc: 0.9726999998092651


## Test Conv2D with DropConnect Wrapper

In [92]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = np.expand_dims(x_train, axis=-1).astype('float32') / 255
x_test = np.expand_dims(x_test, axis=-1).astype('float32') / 255

In [93]:
X = tf.keras.layers.Input(shape=(28, 28, 1))
x = DropConnect(Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid'), prob=0.1)(X)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)
x = DropConnect(Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='valid'), prob=0.1)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPool2D((2,2))(x)

x = Flatten()(x)
x = DropConnect(Dense(units=64, activation="relu"), prob=0.3)(x)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [94]:
model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
drop_connect_21 (DropConnect (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 26, 26, 64)        256       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
drop_connect_22 (DropConnect (None, 11, 11, 128)       73856     
_________________________________________________________________
batch_normalization_3 (Batch (None, 11, 11, 128)       512

In [95]:
# Hyperparameters
batch_size=64
epochs=10


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/10
844/844 [==============================] - 9s 10ms/step - loss: 0.2053 - accuracy: 0.9409 - val_loss: 0.0650 - val_accuracy: 0.9818
Epoch 2/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0613 - accuracy: 0.9819 - val_loss: 0.0553 - val_accuracy: 0.9837
Epoch 3/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0438 - accuracy: 0.9865 - val_loss: 0.0441 - val_accuracy: 0.9873
Epoch 4/10
844/844 [==============================] - 8s 9ms/step - loss: 0.0338 - accuracy: 0.9895 - val_loss: 0.0375 - val_accuracy: 0.9897
Epoch 5/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0247 - accuracy: 0.9925 - val_loss: 0.0393 - val_accuracy: 0.9892
Epoch 6/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0210 - accuracy: 0.9933 - val_loss: 0.0364 - val_accuracy: 0.9900
Epoch 7/10
844/844 [==============================] - 8s 10ms/step - loss: 0.0155 - accuracy: 0.9956 - val_loss: 0.0328 - val_accuracy: 0.9903


In [96]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 0s - loss: 0.0314 - accuracy: 0.9904
Test loss = 0.0314035527408123, Test acc: 0.9904000163078308


## Test LSTM with DropConnect Wrapper

In [14]:
# Load MNIST dataset as NumPy arrays
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [15]:
X = tf.keras.layers.Input(shape=(28,28))

x = DropConnect(LSTM(128, return_sequences=True), prob=0.5)(X)
x = DropConnect(LSTM(128), prob=0.5)(X)
y = Dense(10, activation="softmax")(x)

model = tf.keras.models.Model(X, y)

In [16]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
drop_connect_2 (DropConnect) (None, 128)               80384     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 81,674
Trainable params: 81,674
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Hyperparameters
batch_size=64
epochs=10


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),  # Utilize optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

# Train the network
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs)

Epoch 1/10
844/844 [==============================] - 14s 12ms/step - loss: 1.0768 - accuracy: 0.6540 - val_loss: 0.4349 - val_accuracy: 0.8755
Epoch 2/10
844/844 [==============================] - 9s 11ms/step - loss: 0.3803 - accuracy: 0.8867 - val_loss: 0.2600 - val_accuracy: 0.9222
Epoch 3/10
844/844 [==============================] - 9s 11ms/step - loss: 0.2674 - accuracy: 0.9199 - val_loss: 0.2232 - val_accuracy: 0.9308
Epoch 4/10
844/844 [==============================] - 9s 11ms/step - loss: 0.2119 - accuracy: 0.9359 - val_loss: 0.1669 - val_accuracy: 0.9488
Epoch 5/10
844/844 [==============================] - 9s 11ms/step - loss: 0.1822 - accuracy: 0.9454 - val_loss: 0.1432 - val_accuracy: 0.9560
Epoch 6/10
844/844 [==============================] - 9s 11ms/step - loss: 0.1613 - accuracy: 0.9520 - val_loss: 0.1703 - val_accuracy: 0.9442
Epoch 7/10
844/844 [==============================] - 9s 10ms/step - loss: 0.1461 - accuracy: 0.9556 - val_loss: 0.1227 - val_accuracy: 0.962

In [18]:
# Evaluate the network
print('Evaluate on test data:')
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

Evaluate on test data:
79/79 - 0s - loss: 0.1155 - accuracy: 0.9633
Test loss = 0.11552761495113373, Test acc: 0.9632999897003174
